In [39]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

import bcrypt

In [40]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [41]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [42]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_institution_name(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [43]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-05-17 14:55:28


In [55]:
# open file dialog
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()
file_name

'D:/OneDrive - Ingenio Azucarero Guabira S.A/APP CAÑERO/DATA PARA CARGAR/USUARIOS.xlsx'

In [56]:
data = pd.read_excel(file_name)
data.head(3)

,INST,COD_CA,NOMBRE,AP_PATERNO,AP_MATERNO,CORREO,USUARIO,PASS,TELF
0,82,13302,SABINA,SALO,MARTINEZ,ssalo_13302@gmail.com,13302,123*13302,74647480
1,82,12503,PASCUAL,SALO,SANTOS,psalo_12503@gmail.com,12503,123*12503,71011083
2,82,13025,JUAN BAUTISTA,SALO,SANTOS,jsalo_13025@gmail.com,13025,123*13025,72150289


In [57]:
data.shape

(304, 9)

In [58]:
for i in range(len(data)):
    fila = data.iloc[i]
    # COMPROBAR EXISTENCIA DE USUARIO
    select = 'select * from users where codigo={cod}'.format(cod=fila['COD_CA'])
    # ejecutar seleccion
    select_user = dbQuery(select)
    if len(select_user)>0:
        print('(' + str(i+1) + ')=====USUARIO YA RESTRADO: ', select_user[0]['codigo'] + ' / ' + select_user[0]['name'] + ' ' + select_user[0]['apellido_paterno'])
        continue
    
    # BUSCAR INST
    select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['INST'])
    # ejecutar seleccion
    select_inst = dbQuery(select)
    
    # CREAR CONTARSEÑA
    password = fila['PASS'].encode('utf-8')  # Asegúrese de que la contraseña sea de tipo bytes
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password, salt)
    
    # DATOS DEL USUARIO
    usuario = {
        'codigo' : fila['COD_CA'],
        'nombre' : fila['NOMBRE'],
        'ap_paterno' : fila['AP_PATERNO'],
        'ap_materno' : fila['AP_MATERNO'],
        'correo' : fila['CORREO'],
        'telf' : fila['TELF'],
        'usuario' : fila['USUARIO'],
        'contrasena' : hashed_password.decode('utf-8'),
        'tipo' : 'propietario',
        'inst' : select_inst[0]['id'],
        'abrev' : abbreviate_institution_name(fila['NOMBRE']),
        'estado' : 'activo',
        'usuario_id' : 1,
        'fecha' : formatted_now
    }
    
    # INSERT USER
    query = 'insert into users (name, apellido_paterno, apellido_materno, username, email, codigo, telefono, tipo, estado, slug, password, institucion_id, usuario_id, created_at, updated_at) values (\'{nombre}\', \'{ap_paterno}\', \'{ap_materno}\',\'{username}\', \'{email}\', \'{codigo}\', \'{telefono}\', \'{tipo}\', \'{estado}\', \'{slug}\', \'{password}\', {institucion_id}, {usuario_id}, \'{created_at}\', \'{updated_at}\')'.format(nombre=usuario['nombre'], ap_paterno=usuario['ap_paterno'], ap_materno=usuario['ap_materno'], username=usuario['usuario'], email=usuario['correo'], codigo=usuario['codigo'], telefono=usuario['telf'], tipo=usuario['tipo'], estado=usuario['estado'], slug='', password=usuario['contrasena'], institucion_id=usuario['inst'], usuario_id=usuario['usuario_id'], created_at=usuario['fecha'], updated_at=usuario['fecha'])
    insert_row = dbQuery(query, 1)
    
    # SELECCIONAR EL USUARIO RECIEN CREADO
    select = 'select * from users where codigo={cod}'.format(cod=usuario['codigo'])
    # ejecutar seleccion
    select_user = dbQuery(select)
    
    # GENERAR CODIGO SLUG
    slug_cod = abbreviate_institution_name(select_user[0]['name']) + '-' + str(select_user[0]['id'])
    
    # ACTUALIZAR EL SLUG
    update_slug = 'update users set slug=\'{slug_cod}\' where id={id_user}'.format(slug_cod=slug_cod, id_user=select_user[0]['id'])
    dbQuery(update_slug, 1)
    
    print('(' + str(i+1) + ')USUARIO REGISTRADO: ', str(usuario['codigo']) + " / " + usuario['nombre'] + " " + usuario['ap_paterno'])
    i=i+1

(1)USUARIO REGISTRADO:  13302 / SABINA SALO
(2)USUARIO REGISTRADO:  12503 / PASCUAL SALO
(3)USUARIO REGISTRADO:  13025 / JUAN BAUTISTA SALO
(4)USUARIO REGISTRADO:  16487 / BONIFACIO SALO
(5)USUARIO REGISTRADO:  4808 / VICENTE GUTIERREZ
(6)USUARIO REGISTRADO:  12870 / ANASTACIA SOLIZ
(7)USUARIO REGISTRADO:  13210 / ALFREDO SALO
(8)USUARIO REGISTRADO:  9923 / ARMANDO ONOFRE
(9)=====USUARIO YA RESTRADO:  42075 / LUIS ALFREDO ONOFRE
(10)USUARIO REGISTRADO:  42240 / DIEGO ARMANDO ONOFRE
(11)USUARIO REGISTRADO:  40158 / MARIA LOURDES GUZMAN
(12)USUARIO REGISTRADO:  42175 / JUAN CARLOS MAMANI
(13)USUARIO REGISTRADO:  42176 / GLADYS JANNETTE CAMACHO
(14)USUARIO REGISTRADO:  18077 / RENE RIOS
(15)USUARIO REGISTRADO:  18080 / FELIX CRUZ
(16)USUARIO REGISTRADO:  41919 / HECTOR DANIEL SALAZAR
(17)USUARIO REGISTRADO:  12295 / MARGARITA ROJAS
(18)USUARIO REGISTRADO:  42173 / ADELA ROCHA
(19)USUARIO REGISTRADO:  42174 / ANDRES CLAROS
(20)USUARIO REGISTRADO:  41897 / ARIEL EDSON DELGADILLO
(21)USUARIO

In [10]:
fila = data.iloc[0]
fila['COD_CA']

2710

In [11]:
fila

INST                                 1
COD_CA                            2710
NOMBRE                        JOSELITO
AP_PATERNO                    DEMIQUEL
AP_MATERNO                     JIMENEZ
CORREO        jdemiquel_2710@gmail.com
USUARIO                           2710
PASS                          123*2710
TELF                          70863144
Name: 0, dtype: object

In [12]:
fila['COD_CA']

2710

In [17]:
# seleccionar instituacion
select = 'select * from users where codigo={cod}'.format(cod=fila['COD_CA'])
# ejecutar seleccion
select_user = dbQuery(select)
#select_inst[0]

In [18]:
if len(select_user)==0:
    print('No registrado')
else:
    print('Ya esta registrado')

No registrado


In [19]:
fila

INST                                 1
COD_CA                            2710
NOMBRE                        JOSELITO
AP_PATERNO                    DEMIQUEL
AP_MATERNO                     JIMENEZ
CORREO        jdemiquel_2710@gmail.com
USUARIO                           2710
PASS                          123*2710
TELF                          70863144
Name: 0, dtype: object

In [20]:
# BUSCAR INST
select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['INST'])
# ejecutar seleccion
select_inst = dbQuery(select)
# si es mayor a cero, la inst ya esta registrada
select_inst[0]

{'id': 26,
 'codigo': '1',
 'nombre': '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.',
 'slug': '1-asociacion-de-pequenos-productores-de-cana-campesino-ctc-26',
 'direccion': 'C/ Rosendo Paz Nº 148',
 'telefono': '9220257',
 'estado': 'activo',
 'created_at': datetime.datetime(2023, 5, 16, 14, 43, 40),
 'updated_at': datetime.datetime(2023, 5, 16, 14, 43, 40)}

In [21]:
# CREAR CONTARSEÑA
password = fila['PASS'].encode('utf-8')  # Asegúrese de que la contraseña sea de tipo bytes
salt = bcrypt.gensalt()
hashed_password = bcrypt.hashpw(password, salt)
print(hashed_password)

b'$2b$12$XvKNFG29MmuFSWDFYp2UGuX22R46t0Xs7Sec35tX9sV5K9jef9p9y'


In [23]:
# DATOS DEL USUARIO
usuario = {
    'codigo' : fila['COD_CA'],
    'nombre' : fila['NOMBRE'],
    'ap_paterno' : fila['AP_PATERNO'],
    'ap_materno' : fila['AP_MATERNO'],
    'correo' : fila['CORREO'],
    'telf' : fila['TELF'],
    'usuario' : fila['USUARIO'],
    'contrasena' : hashed_password.decode('utf-8'),
    'tipo' : 'propietario',
    'inst' : select_inst[0]['id'],
    'abrev' : abbreviate_institution_name(fila['NOMBRE']),
    'estado' : 'activo',
    'usuario_id' : 1,
    'fecha' : formatted_now
}

In [24]:
query = 'insert into users (name, apellido_paterno, apellido_materno, username, email, codigo, telefono, tipo, estado, slug, password, institucion_id, usuario_id, created_at, updated_at) values (\'{nombre}\', \'{ap_paterno}\', \'{ap_materno}\',\'{username}\', \'{email}\', \'{codigo}\', \'{telefono}\', \'{tipo}\', \'{estado}\', \'{slug}\', \'{password}\', {institucion_id}, {usuario_id}, \'{created_at}\', \'{updated_at}\')'.format(nombre=usuario['nombre'], ap_paterno=usuario['ap_paterno'], ap_materno=usuario['ap_materno'], username=usuario['usuario'], email=usuario['correo'], codigo=usuario['codigo'], telefono=usuario['telf'], tipo=usuario['tipo'], estado=usuario['estado'], slug='', password=usuario['contrasena'], institucion_id=usuario['inst'], usuario_id=usuario['usuario_id'], created_at=usuario['fecha'], updated_at=usuario['fecha'])
insert_row = dbQuery(query, 1)

In [25]:
# SELECCIONAR EL USUARIO RECIEN CREADO
select = 'select * from users where codigo={cod}'.format(cod=usuario['codigo'])
# ejecutar seleccion
select_user = dbQuery(select)

In [26]:
# GENERAR CODIGO SLUG
slug_cod = abbreviate_institution_name(select_user[0]['name']) + '-' + str(select_user[0]['id'])
slug_cod

'joselito-70'

In [27]:
# ACTUALIZAR EL SLUG
update_slug = 'update users set slug=\'{slug_cod}\' where id={id_user}'.format(slug_cod=slug_cod, id_user=select_user[0]['id'])
dbQuery(update_slug, 1)

()